## Resource-Constrained Project Scheduling Problem with Trasfer Times

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)).

### Problem Definiton

The Resource-Constrained Project Scheduling Problem with Trasfer Times (RCPSPTT) is an extension of the RCPSP problem defined. In RCPSP, activities are executed using available resources, precedence relationships between them must be adhered to, and the total makespan is minimized. Compared to the RCPSP, RCPSPTT introduces transfer times between activities. This means that resources do not have to be directly available after the activity that uses them finishes; rather, a transfer of resources to the following activity might be required. The transfer time depends on both the activities and the resource transferred.

Formally, the RCPSPTT problem is defined as follows:

Let $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$, $\ \hat{A}\in\mathbb{N}$, be a set of activities with indices $i,j\in\mathcal{A}$. (RCPSP)

Let $P\in\mathbb{N}^{|\mathcal{A}|}$ be a vector of activity processing times. (RCPSP)

Let $\mathcal{E}=\{(i,j)\mid \text{activity } i \text{ precedes activity } j\}$ be a set of precedences. (RCPSP)

Let $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$, $\ \hat{R}\in\mathbb{N}$, be a set of primary resources indexed by $r\in\mathcal{R}$. (RCPSP)

Let $C\in\mathbb{N}^{|\mathcal{R}|}$ be a vector of primary resource capacities. (RCPSP)

Let $\mathbf{Q}\in\mathbb{Z}_{{\ge 0}}^{|\mathcal{A}|\times|\mathcal{R}|}$ be the activity–resource consumption matrix, with entries $\mathbf{Q}_{i,r}$ denoting the consumption of resource $r$ by activity $i$ ($i=0 \Rightarrow \mathbf{Q}_{i,r}=C_r$). (RCPSP)

Let $\Delta\in\mathbb{Z}_{\ge 0}^{|\mathcal{A}|\times|\mathcal{A}|\times|\mathcal{R}|}$ be a matrix of transfer delays, with entries $\Delta_{i,j,r}$ denoting the delay required if resource $r$ is transferred between activities $i$ and $j$. (RCPSPTT)

### Aditional Resources

- **Instances for RCPSPTT**
  - https://www2.informatik.uni-osnabrueck.de/kombopt/data/rcpsp/

- **For image of the problem instance see this article:** [An efficient genetic algorithm to solve the resource-constrained project scheduling problem with transfer times](https://www.sciencedirect.com/science/article/pii/S0377221717306549)

- [Problem defintion](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)